# Primary Import and Export Countries by State

Following the announcement by the new administration to raise tariffs on Canada, Mexico, and China (and the not entirely unexpected retaliatory measures from those targeted countries), it would be interesting to know which countries are the primary sources and destinations for imports and exports.

This data can be obtained from the US government’s International Trade Administration at trade.gov however, it is not easily obtained since the APIs made available to the public do not have any endpoints to access this type of data. Instead, we will have to resort to scraping the data from where it is available – a web page located at:

[https://tsereports.trade.gov/views/StateAnnualDashboardLive_16421978287720/StateTradebyProduct?%3AshowVizHome=no&%3Aembed=true&%3Atoolbar=yes%2Ctop](https://tsereports.trade.gov/views/StateAnnualDashboardLive_16421978287720/StateTradebyProduct?%3AshowVizHome=no&%3Aembed=true&%3Atoolbar=yes%2Ctop)

As is often the case with scraping web pages, the data is buried deep within the structure making it unfeasible to use quick and dirty methods like grabbing a table from a webpage using native Pandas functionality. Furthermore, the data on this page is displayed only a single state at a time and requires interacting with the webpage to modify filters. Selenium to the rescue!

After acquiring the data, we also want to display it graphically using a map of the USA to show each state’s dependence on their primary export and import partners. For this, we will employ Choropleth maps.

In [1]:
import requests

api_key = "ENTERYOURAPIKEYHERE"

The following attempt fails because even though the variables used come from the documented set of variables at [https://api.census.gov/data/timeseries/intltrade/exports/enduse/variables.html](https://api.census.gov/data/timeseries/intltrade/exports/enduse/variables.html) and otherwise accessing the API directly as an https query using [https://api.census.gov/data/timeseries/intltrade/exports/enduse?get=ALL_VAL_YR,DISTRICT,DIST_NAME&time=2023&key=YOUR_API_KEY_HERE](https://api.census.gov/data/timeseries/intltrade/exports/enduse?get=ALL_VAL_YR,DISTRICT,DIST_NAME&time=2023&key=YOUR_API_KEY_HERE), the programmatic API call fails. In any case, it doesn't provide the state by state export totals nor the destination countries.

So instead, we will use selenium with chromedriver to access the page located at [https://www.trade.gov/data-visualization/tradestats-express-state-trade-product](https://www.trade.gov/data-visualization/tradestats-express-state-trade-product) although that page is simply a container for n iframe whose contents are actually located at [https://tsereports.trade.gov/views/StateAnnualDashboardLive_16421978287720/StateTradebyProduct?%3AshowVizHome=no&%3Aembed=true&%3Atoolbar=yes%2Ctop](https://tsereports.trade.gov/views/StateAnnualDashboardLive_16421978287720/StateTradebyProduct?%3AshowVizHome=no&%3Aembed=true&%3Atoolbar=yes%2Ctop)

For testing the validity of XPath expressions to extract content using Selenium, we can use JavaScript code

For example, on the page https://www.cnn.com/election/2024/results/florida

```js
counties_link = document.evaluate('//a[text()="See county results"]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null);
counties_link.singleNodeValue;
```

After navigating to the County results, use additional JavaScript code to find the table we want to access, and extract the values from it that we need

For example, on the page https://www.cnn.com/election/2024/results/florida/president

```js
curr_state_votes = []

rows = document.evaluate('//h2[text()="Results by county"]/following-sibling::div/table/tbody/tr', document, null, XPathResult.ORDERED_NODE_SNAPSHOT_TYPE, null);

for(r=0, r<rows.snapshotLength, r++) {
    curr_row = rows.snapshotItem(r);
    county_name = document.evaluate('th/div/text()', curr_row, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.data;
    curr_cells = document.evaluate('td', curr_row, null, XPathResult.ORDERED_NODE_SNAPSHOT_TYPE, null);

    curr_dem = curr_cells.snapshotItem(0);
    curr_dem_divs = document.evaluate('div', curr_dem, null, XPathResult.ORDERED_NODE_SNAPSHOT_TYPE, null);
    curr_dem_percent = (parseFloat(curr_dem_divs.snapshotItem(0).innerText) / 100.0).toFixed(3);
    curr_dem_votecnt = parseInt(curr_dem_divs.snapshotItem(1).innerText.replace(",", ""));

    curr_rep = curr_cells.snapshotItem(curr_cells.snapshotLength - 1);
    curr_rep_divs = document.evaluate('div', curr_rep, null, XPathResult.ORDERED_NODE_SNAPSHOT_TYPE, null);
    curr_rep_percent = (parseFloat(curr_rep_divs.snapshotItem(0).innerText) / 100.0).toFixed(3);
    curr_rep_votecnt = parseInt(curr_rep_divs.snapshotItem(1).innerText.replace(",", ""));

    curr_state_votes.push({statename: "florida", countyname: county_name, dempercent: curr_dem_percent, demvotecnt: curr_dem_votecnt, reppercent: curr_rep_percent, repvotecnt: curr_rep_votecnt});
}
```

In [2]:
# Import the libraries we will use to scrape symbol data
import io
import os
import glob
import re
import csv

from chromedriver_py import binary_path # This gets the path to the chromedriver executable
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import time
from datetime import datetime, timedelta, timezone
import pandas as pd

# For plotting USA map
import plotly.express as px
import kaleido

The above also will work only partially because the trade website uses Tableau for generating these reports and the value of the exports, although displayed, cannot be obtained by using XPath (possibly generated on the fly with JavaScript?). So, we can get the ranking of export destination, but without the value of the exports.

In [3]:
# Let's check where the chromedriver executable was installed

binary_path

'C:\\Users\\cesin\\AppData\\Roaming\\Python\\Python311\\site-packages\\chromedriver_py\\chromedriver_win64.exe'

In [4]:
CHROMEDRIVER_USER_DIR_PATH = "/".join(binary_path.split("\\")[:-1])
CHROMEDRIVER_USER_DIR_PATH

'C:/Users/cesin/AppData/Roaming/Python/Python311/site-packages/chromedriver_py'

In [5]:
# Setup our timeouts

timeout = 30
pause = 3

In [6]:
# We need a translation dictionary from state name to state abbreviation (for mapping)

dic_states = {
	  "Alabama": "AL"
	, "Alaska": "AK"
	, "Arizona": "AZ"
	, "Arkansas": "AR"
	, "California": "CA"
	, "Colorado": "CO"
	, "Connecticut": "CT"
	, "Delaware": "DE"
	, "District of Columbia": "DC"
	, "Florida": "FL"
	, "Georgia": "GA"
	, "Hawaii": "HI"
	, "Idaho": "ID"
	, "Illinois": "IL"
	, "Indiana": "IN"
	, "Iowa": "IA"
	, "Kansas": "KS"
	, "Kentucky": "KY"
	, "Louisiana": "LA"
	, "Maine": "ME"
	, "Maryland": "MD"
	, "Massachusetts": "MA"
	, "Michigan": "MI"
	, "Minnesota": "MN"
	, "Mississippi": "MS"
	, "Missouri": "MO"
	, "Montana": "MT"
	, "Nebraska": "NE"
	, "Nevada": "NV"
	, "New Hampshire": "NH"
	, "New Jersey": "NJ"
	, "New Mexico": "NM"
	, "New York": "NY"
	, "North Carolina": "NC"
	, "North Dakota": "ND"
	, "Ohio": "OH"
	, "Oklahoma": "OK"
	, "Oregon": "OR"
	, "Pennsylvania": "PA"
    , "Puerto Rico": "PR"
	, "Rhode Island": "RI"
	, "South Carolina": "SC"
	, "South Dakota": "SD"
	, "Tennessee": "TN"
	, "Texas": "TX"
	, "Utah": "UT"
	, "Vermont": "VT"
	, "Virginia": "VA"
	, "Washington": "WA"
	, "West Virginia": "WV"
	, "Wisconsin": "WI"
	, "Wyoming": "WY"
}

In [7]:
chrome_options = Options()
# See list of options/switches for Chrome here:
# https://peter.sh/experiments/chromium-command-line-switches/
#chrome_options.add_argument("--headless") # Run with any UI
chrome_options.add_argument("--start-maximized") # Maximize the window to fit to the screen
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--user-data-dir=" + CHROMEDRIVER_USER_DIR_PATH)
#driver = webdriver.Chrome(executable_path=CHROMEDRIVER_EXECUTABLE_PATH, chrome_options=chrome_options)
webdriver.ChromeOptions = chrome_options
driver = webdriver.Chrome()
driver.set_window_size(1920, 1080)

In [8]:
# Set the base URL

url = "https://tsereports.trade.gov/views/StateAnnualDashboardLive_16421978287720/StateTradebyProduct?%3AshowVizHome=no&%3Aembed=true&%3Atoolbar=yes%2Ctop"

In [9]:
# Load the initial web page from the base URL

wait = WebDriverWait(driver, timeout)
driver.get(url)
#WebDriverWait(driver=driver, timeout=timeout).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="tab-tvView tvimagesNS" and @role="region"]')))
WebDriverWait(driver=driver, timeout=timeout).until(EC.visibility_of_element_located((By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_2"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')))

<selenium.webdriver.remote.webelement.WebElement (session="4f044bb928b6851c9221254a88bb0b4f", element="f.1C24F95FC37A57790F80DCEC4FB0020F.d.689BBAC1ADA82ACF2E4E6D30F054C987.e.44")>

In [10]:
years_since = driver.find_element(By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_2"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')
years_since

<selenium.webdriver.remote.webelement.WebElement (session="4f044bb928b6851c9221254a88bb0b4f", element="f.1C24F95FC37A57790F80DCEC4FB0020F.d.689BBAC1ADA82ACF2E4E6D30F054C987.e.44")>

In [11]:
# Display the oldest year in the data being displayed
#  
years_since.text

'2017'

In [12]:
# Click on the year combobox to change to the most recent year (as of this writing, 2023)

years_since.click()

In [13]:
# Bty clicking ont he current year, it will open the combobox allowing us to select the target year

years_combo = driver.find_element(By.XPATH, '//div[@role="menu" and @aria-label="Start Year:"]')

In [14]:
# Locate the combobox item representing the year 2023

year_2023 = years_combo.find_element(By.XPATH, './div[@class="tabMenuContent"]//span[text() = "2023"]')
year_2023.click()

# Force Python to sleep (note that this is necessary as the drive wait doesn't pause execution)
time.sleep(pause)

In [15]:
# Set up our dictionary for export countries by state
state_exports = {}

# Loop until the last state is clicked
while True:

    # Get the list of countries being displayed and choose the 1st item (the 0th item is a header)
    top_countries = driver.find_elements(By.XPATH, '//div[@class="tab-tvYLabel tvimagesNS"]/div[@class="tab-ReactView"]/div[@class="tab-vizLeftSceneMargin"]//div[@class="tab-vizHeaderWrapper"]')
    curr_export_dest = top_countries[1].text

    # Delete the overlaid div that prevents the element from being clicked
    delete_elements = driver.find_elements(By.XPATH, '//div[@class="wcGlassPane"]') # id="loadingGlassPane"
    if len(delete_elements) > 0:
        driver.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
        """, delete_elements[0])

    # Wait until the JavaScript code deletes the element
    #wait = WebDriverWait(driver, pause)
    time.sleep(pause)

    # Locate the currently displayed state and click it (this will open the combobox of states again)
    WebDriverWait(driver=driver, timeout=timeout).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_1"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')))
    curr_state = driver.find_element(By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_1"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')
    curr_state.click()
    
    # Locate the State combobox
    state_combo = driver.find_element(By.XPATH, '//div[@role="menu" and @aria-label="State:"]')

    # Save the name of the currently selected state and use it to add an entry into the dictionary
    curr_state_text = curr_state.text
    state_exports[dic_states[curr_state_text]] = [curr_state_text, curr_export_dest]

    # Now find the next element in the combobox based on the currently selected coombobox element (use following-sibling to navigate to the next one)
    state_curr = state_combo.find_element(By.XPATH, f'./div[@class="tabMenuContent"]//span[text() = "{curr_state.text}"]')
    state_next = state_curr.find_elements(By.XPATH, f'../../following-sibling::*')

    # Break out of the loop once the last state has been reached and there are no "following-siblings"
    if len(state_next) < 1:
        state_curr.click() # Close the combo box
        break

    # Since there are still some states remaining that we haven't visited yet, click on the next state name
    state_next[0].click()

    # Pause to allow the screen to reload with new data pertinent to the state
    time.sleep(pause)

In [16]:
# Now, let's do the imports

# Click the radio button to change from Exports to Imports
rdo_imports = driver.find_element(By.XPATH, '//a[text()="Imports"]/preceding-sibling::input[@type="radio"]')
rdo_imports.click()
time.sleep(pause)

In [17]:
# Open the combo box of state names
state_control = driver.find_element(By.XPATH, '//h3[@title="State:"]/../../../div/div[@class="PCContent"]/span')
state_control.click()
time.sleep(pause)

In [18]:
 # Locate the State combobox
state_combo = driver.find_element(By.XPATH, '//div[@role="menu" and @aria-label="State:"]')

In [19]:
# Get the list of states, and click on the first one (Alabama)
all_states = state_combo.find_elements(By.XPATH, f'./div[@class="tabMenuContent"]//span')
print(len(all_states))
all_states[0].click()
time.sleep(pause)

52


In [20]:
# Set up our dictionary for import countries by state
state_imports = {}

# Loop until the last state is clicked
while True:

    # Get the list of countries being displayed and choose the 1st item (the 0th item is a header)
    top_countries = driver.find_elements(By.XPATH, '//div[@class="tab-tvYLabel tvimagesNS"]/div[@class="tab-ReactView"]/div[@class="tab-vizLeftSceneMargin"]//div[@class="tab-vizHeaderWrapper"]')
    curr_import_dest = top_countries[1].text

    # Delete the overlaid div that prevents the element from being clicked
    delete_elements = driver.find_elements(By.XPATH, '//div[@class="wcGlassPane"]') # id="loadingGlassPane"
    if len(delete_elements) > 0:
        driver.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
        """, delete_elements[0])

    # Wait until the JavaScript code deletes the element
    #wait = WebDriverWait(driver, pause)
    time.sleep(pause)

    # Locate the currently displayed state and click it (this will open the combobox of states again)
    WebDriverWait(driver=driver, timeout=timeout).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_1"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')))
    curr_state = driver.find_element(By.XPATH, '//div[@id="tableau_base_widget_ParameterControl_1"]//div[@class="tabComboBoxNameContainer tab-ctrl-formatted-fixedsize"]/span')
    curr_state.click()
    
    # Locate the State combobox
    state_combo = driver.find_element(By.XPATH, '//div[@role="menu" and @aria-label="State:"]')

    # Save the name of the currently selected state and use it to add an entry into the dictionary
    curr_state_text = curr_state.text
    state_imports[dic_states[curr_state_text]] = [curr_state_text, curr_import_dest]

    # Now find the next element in the combobox based on the currently selected coombobox element (use following-sibling to navigate to the next one)
    state_curr = state_combo.find_element(By.XPATH, f'./div[@class="tabMenuContent"]//span[text() = "{curr_state.text}"]')
    state_next = state_curr.find_elements(By.XPATH, f'../../following-sibling::*')

    # Break out of the loop once the last state has been reached and there are no "following-siblings"
    if len(state_next) < 1:
        state_curr.click() # Close the combo box
        break

    # Since there are still some states remaining that we haven't visited yet, click on the next state name
    state_next[0].click()

    # Pause to allow the screen to reload with new data pertinent to the state
    time.sleep(pause)

In [21]:
# Let's look at the data we collected for export countries

list(state_exports.items())[:10]

[('AL', ['Alabama', 'Germany']),
 ('AK', ['Alaska', 'China']),
 ('AZ', ['Arizona', 'Mexico']),
 ('AR', ['Arkansas', 'Canada']),
 ('CA', ['California', 'Mexico']),
 ('CO', ['Colorado', 'Canada']),
 ('CT', ['Connecticut', 'Canada']),
 ('DE', ['Delaware', 'Canada']),
 ('DC', ['District of Columbia', 'United Kingdom']),
 ('FL', ['Florida', 'Canada'])]

In [22]:
# Let's look at the data we collected for import countries

list(state_imports.items())[:10]

[('AL', ['Alabama', 'Mexico']),
 ('AK', ['Alaska', 'South Korea']),
 ('AZ', ['Arizona', 'Mexico']),
 ('AR', ['Arkansas', 'Canada']),
 ('CA', ['California', 'China']),
 ('CO', ['Colorado', 'Canada']),
 ('CT', ['Connecticut', 'Canada']),
 ('DE', ['Delaware', 'Mexico']),
 ('DC', ['District of Columbia', 'Australia']),
 ('FL', ['Florida', 'China'])]

In [23]:
driver.quit()

In [24]:
df_exports = pd.DataFrame.from_dict(state_exports, columns=["State", "Destination"], orient="index")
#df_exports = pd.DataFrame.from_dict(abbrev_exports, columns=["State", "Destination"], orient="index")
df_exports.index.names = ["StateAbbr"]
df_exports.reset_index(drop=False, inplace=True)
# Since Puerto Rico doesn't appear on the map, we will drop it from the dataframe
df_exports = df_exports[df_exports["StateAbbr"] != "PR"]
df_exports.head(10)

,StateAbbr,State,Destination
0,AL,Alabama,Germany
1,AK,Alaska,China
2,AZ,Arizona,Mexico
3,AR,Arkansas,Canada
4,CA,California,Mexico
5,CO,Colorado,Canada
6,CT,Connecticut,Canada
7,DE,Delaware,Canada
8,DC,District of Columbia,United Kingdom
9,FL,Florida,Canada


In [25]:
fig = px.choropleth(
      df_exports
    , locations = "StateAbbr"
    , locationmode = "USA-states"
    , color = "Destination"
    , scope = "usa"
    , hover_data = {"StateAbbr": False, "State": True, "Destination": True}
    #, labels = {"Destination": "Target", "State":""}
    , title = "Primary Export Country by State"
    #, width = 1024
    #, height = 768
    , category_orders = {"Destination": sorted(pd.unique(df_exports["Destination"]))}
    #, color_discrete_sequence = px.colors.qualitative.Alphabet[1:]
    , color_discrete_sequence = px.colors.qualitative.Light24
)

fig.update_layout(
      width = 1000
    , height = 500
    , margin = {"r": 0, "t": 50, "l": 0, "b": 0}
)

#fig.show()
fig.write_image("export_countries_by_state.png")

![export_countries_by_state.png](export_countries_by_state.png)

In [26]:
df_imports = pd.DataFrame.from_dict(state_imports, columns=["State", "Source"], orient="index")
#df_imports = pd.DataFrame.from_dict(abbrev_imports, columns=["State", "Source"], orient="index")
df_imports.index.names = ["StateAbbr"]
df_imports.reset_index(drop=False, inplace=True)
# Since Puerto Rico doesn't appear on the map, we will drop it from the dataframe
df_imports = df_imports[df_imports["StateAbbr"] != "PR"]
df_imports.head(10)

,StateAbbr,State,Source
0,AL,Alabama,Mexico
1,AK,Alaska,South Korea
2,AZ,Arizona,Mexico
3,AR,Arkansas,Canada
4,CA,California,China
5,CO,Colorado,Canada
6,CT,Connecticut,Canada
7,DE,Delaware,Mexico
8,DC,District of Columbia,Australia
9,FL,Florida,China


In [27]:
fig = px.choropleth(
      df_imports
    , locations = "StateAbbr"
    , locationmode = "USA-states"
    , color = "Source"
    , scope = "usa"
    , hover_data = {"StateAbbr": False, "State": True, "Source": True}
    #, labels = {"Source": "Target", "State":""}
    , title = "Primary Import Country by State"
    #, width = 1024
    #, height = 768
    , category_orders = {"Source": sorted(pd.unique(df_imports["Source"]))}
    #, color_discrete_sequence = px.colors.qualitative.Alphabet[1:]
    , color_discrete_sequence = px.colors.qualitative.Light24
)

fig.update_layout(
      width = 1000
    , height = 500
    , margin = {"r": 0, "t": 50, "l": 0, "b": 0}
)

#fig.show()
fig.write_image("import_countries_by_state.png")

![import_countries_by_state.png](import_countries_by_state.png)